# Tools Agents

In [13]:
import os
from dotenv import load_dotenv

load_dotenv()

False

In [ ]:
os.environ['HF_TOKEN']=os.getenv('HF_TOKEN')

In [1]:
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [5]:
api_wrapper_wikipedia = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)

In [6]:
wikipedia=WikipediaQueryRun(api_wrapper=api_wrapper_wikipedia)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)

In [4]:
wikipedia.name

'wikipedia'

In [7]:
arxiv.name

'arxiv'

In [8]:
tools=[wikipedia,arxiv]

In [9]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/home/iceyisaak/anaconda3/envs/langchain-chatbot-search-engine-tools-agents/lib/python3.11/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250))]

---

## Custom Tools - RAG Tools

In [14]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

In [16]:
webbase_loader = WebBaseLoader("https://docs.langchain.com/")
docs=webbase_loader.load()
splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
chunks=splitter.split_documents(docs)
huggingface_embeddings=HuggingFaceEmbeddings()
vector_store=FAISS.from_documents(chunks,huggingface_embeddings)
retriever=vector_store.as_retriever()

retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x738bd0235950>, search_kwargs={})

### Retriever Tool

In [18]:
from langchain_core.tools.retriever import create_retriever_tool

In [19]:
retriever_tool=create_retriever_tool(retriever,"langsmith-search","Search any info about Langsmith")

In [20]:
retriever_tool

StructuredTool(name='langsmith-search', description='Search any info about Langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=<function create_retriever_tool.<locals>.func at 0x738bd00e45e0>, coroutine=<function create_retriever_tool.<locals>.afunc at 0x738bd00e4b80>)

---

In [21]:
tools=[wikipedia,arxiv,retriever_tool]

In [22]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/home/iceyisaak/anaconda3/envs/langchain-chatbot-search-engine-tools-agents/lib/python3.11/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 StructuredTool(name='langsmith-search', description='Search any info about Langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=<function create_retriever_tool.<locals>.func at 0x738bd00e45e0>, coroutine=<function create_retriever_tool.<locals>.afunc at 0x738bd00e4b80>)]

---